### Task description

### Tasks:

+ Prepare shape maps of agricultural (i.e., crop) areas in the United Arab Emirates.
+ Based on the prepared shape maps, write Python code using Google Earth Engine API to extract the relevant Landsat and Sentinel satellite bands and images across different timeframes.
Source images: https://geodata.lib.utexas.edu/catalog/nyu-2451-44247

In [4]:
%ls

extract_satellite_images.ipynb  nyu_2451_44247/  nyu_2451_44247.zip


In [18]:
import ee

# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize()

Enter verification code: 4/1AWtgzh7Vcmrd_BcO3gPk2oYTaAtPSfY80eKBm1YLJnkKmYNK-6aWQhpvCx4

Successfully saved authorization token.


In [20]:
# Load the shape map as a feature collection
shp = ee.FeatureCollection("projects/ee-images/assets/palmtrees")

# Define a date range
start_date = '2018-01-01'
end_date = '2018-12-31'

# Load the Landsat collection, select relevant bands, and filter by date range and shape
landsat_collection = ee.ImageCollection("LANDSAT/LC08/C01/T1_SR") \
    .select(['B4', 'B3', 'B2']) \
    .filterDate(start_date, end_date) \
    .filterBounds(shp)

# Load the Sentinel-2 collection, select relevant bands, and filter by date range and shape
sentinel_collection = ee.ImageCollection("COPERNICUS/S2") \
    .select(['B4', 'B3', 'B2']) \
    .filterDate(start_date, end_date) \
    .filterBounds(shp)

# Print the number of images in each collection
print("Number of Landsat images: ", landsat_collection.size().getInfo())
print("Number of Sentinel-2 images: ", sentinel_collection.size().getInfo())

# Select the first image in each collection
landsat_image = landsat_collection.first()
sentinel_image = sentinel_collection.first()

# Display the images
print("First Landsat Image:")
display(landsat_image)
print("First Sentinel-2 Image:")
display(sentinel_image)


Number of Landsat images:  222
Number of Sentinel-2 images:  1696
First Landsat Image:


First Sentinel-2 Image:


### Export a single image to Drive

In [35]:
## Import the download module
from ee import batch

# Define the task for downloading the image
task1 = ee.batch.Export.image.toDrive(landsat_image, 
                                     description="Landsat_image_download",
                                     folder="GEE_downloads",
                                     scale=100, 
                                     region=shp.geometry().bounds().getInfo()['coordinates'])

# Start the task
task1.start()

# Check task status
task1.status()

{'state': 'READY',
 'description': 'Landsat_image_download',
 'creation_timestamp_ms': 1675100886297,
 'update_timestamp_ms': 1675100886297,
 'start_timestamp_ms': 0,
 'task_type': 'EXPORT_IMAGE',
 'id': 'PATZHXZHND25LBERQMOSK7V2',
 'name': 'projects/earthengine-legacy/operations/PATZHXZHND25LBERQMOSK7V2'}

In [36]:
# Repeat the same process for the Sentinel-2 image
task2 = ee.batch.Export.image.toDrive(sentinel_image, 
                                     description="Sentinel_image_download",
                                     folder="GEE_downloads",
                                     scale=100, 
                                     region=shp.geometry().bounds().getInfo()['coordinates'])
task2.start()
task2.status()

{'state': 'READY',
 'description': 'Sentinel_image_download',
 'creation_timestamp_ms': 1675101003496,
 'update_timestamp_ms': 1675101003496,
 'start_timestamp_ms': 0,
 'task_type': 'EXPORT_IMAGE',
 'id': '2IVMVSJMAYHWT2YLLMLMW6QP',
 'name': 'projects/earthengine-legacy/operations/2IVMVSJMAYHWT2YLLMLMW6QP'}

In [34]:
task.status()

{'state': 'CANCEL_REQUESTED',
 'description': 'Sentinel_image_download',
 'creation_timestamp_ms': 1675099919310,
 'update_timestamp_ms': 1675100291479,
 'start_timestamp_ms': 1675099926154,
 'task_type': 'EXPORT_IMAGE',
 'attempt': 1,
 'id': 'C65S7NWWYDPHZP5FWPSRUZTH',
 'name': 'projects/earthengine-legacy/operations/C65S7NWWYDPHZP5FWPSRUZTH'}

### Export multiple images to drive

In [ ]:
# Export Landsat images to Google Drive
for image in landsat_collection.toList(1000):
    ee.batch.Export.image.toDrive(image=ee.Image(image), **param).start()

In [ ]:
# Export Sentinel images to Google Drive
for image in sentinel_collection.toList(1000):
    ee.batch.Export.image.toDrive(image=ee.Image(image), **param).start()